[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# Deployment

## Review 

メモリを持つエージェントまで構築しました：

* `act` - モデルに特定のツールを呼び出させる
* `observe` - ツールの出力をモデルに戻す
* `reason` - モデルにツール出力について推論させ、次に何をするかを決定させる（例：別のツールを呼び出すか、直接応答するか）
* `persist state` - インメモリチェックポインターを使用して、中断を伴う長時間の会話をサポート

## Goals

これで、エージェントをローカルでStudioに、そして`LangGraph Cloud`に実際にデプロイする方法について説明します。

In [3]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## Concepts

理解すべき中心的な概念がいくつかあります -

`LangGraph` —
- PythonとJavaScriptライブラリ
- エージェントワークフローの作成を可能にする

`LangGraph API` —
- グラフコードをバンドル
- 非同期操作を管理するためのタスクキューを提供
- インタラクション間でstateを維持するための永続化を提供

`LangGraph Cloud` --
- LangGraph APIのホステッドサービス
- GitHubリポジトリからのグラフのデプロイを可能にする
- また、デプロイされたグラフの監視とトレーシングを提供
- 各デプロイメントの一意のURLを介してアクセス可能

`LangGraph Studio` --
- LangGraphアプリケーション用の統合開発環境（IDE）
- APIをバックエンドとして使用し、グラフのリアルタイムテストと探索を可能にする
- ローカルまたはクラウドデプロイメントで実行可能

`LangGraph SDK` --
- LangGraphグラフと programmatically 対話するためのPythonライブラリ
- ローカルまたはクラウドで提供されるグラフと作業するための一貫したインターフェースを提供
- クライアントの作成、アシスタントへのアクセス、スレッド管理、実行の実行を可能にする

## Testing Locally

**⚠️ 免責事項**

これらのビデオの撮影以降、Studioを更新してローカルで実行し、ブラウザで開けるようになりました。これが（ビデオで示されているDesktopアプリを使用するのではなく）Studioを実行する推奨方法になりました。ローカル開発サーバーに関するドキュメントは[こちら](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)と[こちら](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)をご覧ください。ローカル開発サーバーを開始するには、このモジュールの`/studio`ディレクトリでターミナルで次のコマンドを実行します：

```
langgraph dev
```

次の出力が表示されるはずです：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

ブラウザを開き、Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`に移動してください。

In [1]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [2]:
from langgraph_sdk import get_client

In [3]:
# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [4]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-03-04T22:57:28.424565+00:00',
 'updated_at': '2025-03-04T22:57:28.424565+00:00',
 'version': 1}

In [3]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

これで、[`client.runs.stream`](https://langchain-ai.github.io/langgraph/concepts/low_level/#stream-and-astream)でエージェントを実行できます：

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

ストリーミングについては、将来のモジュールで詳しく説明します。

今のところ、`stream_mode="values"`でグラフの各ステップ後にstateの完全な値を[ストリーミング](https://langchain-ai.github.io/langgraph/cloud/how-tos/stream_values/)していることを認識してください。
 
stateは`chunk.data`でキャプチャされます。

In [4]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'cdbd7bd8-c476-4ad4-8ab7-4ad9e3654267', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5'}, 'type': 'ai', 'name': None, 'id': 'run-06c7243c-426d-4c81-a113-f1335dda5fb2', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_iIPryzZZxRtXozwwhVtFObNO', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '988cb170-f6e6-43c1-82fd-309f519abe6d', 'tool_call_id': 'c

## Testing with Cloud

[こちら](https://langchain-ai.github.io/langgraph/cloud/quick_start/#deploy-from-github-with-langgraph-cloud)で説明されているように、LangSmithを介してCloudにデプロイできます。

### GitHubで新しいリポジトリを作成

* GitHubアカウントに移動
* 右上角の「+」アイコンをクリックし、`"New repository"`を選択
* リポジトリに名前を付ける（例：`langchain-academy`）

### GitHubリポジトリをリモートとして追加

* このコースの開始時に`langchain-academy`をクローンしたターミナルに戻る
* 新しく作成したGitHubリポジトリをリモートとして追加

```
git remote add origin https://github.com/your-username/your-repo-name.git
```
* プッシュする
```
git push -u origin main
```

### LangSmithをGitHubリポジトリに接続

* [LangSmith](hhttps://smith.langchain.com/)に移動
* 左のLangSmithパネルの`deployments`タブをクリック
* `+ New Deployment`を追加
* 次に、コース用に作成したGithubリポジトリ（例：`langchain-academy`）を選択
* `LangGraph API config file`を`studio`ディレクトリの1つに向ける
* 例えば、module-1の場合：`module-1/studio/langgraph.json`
* APIキーを設定する（例：`module-1/studio/.env`ファイルからコピーするだけ）

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### デプロイメントでの作業

その後、いくつかの異なる方法でデプロイメントと対話できます：

* 前と同様に[SDK](https://langchain-ai.github.io/langgraph/cloud/quick_start/#use-with-the-sdk)で。
* [LangGraph Studio](https://langchain-ai.github.io/langgraph/cloud/quick_start/#interact-with-your-deployment-via-langgraph-studio)で。

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

ここのnotebookでSDKを使用するには、`LANGSMITH_API_KEY`が設定されていることを確認してください！

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c